# <center>Generating Music Using LSTM Cells</center>

This workbook will implement modified code from (this)[https://github.com/corynguyen19/midi-lstm-gan] GitHub repo.

The idea is to read in MIDI files and convert them to arrays of notes. Then an RNN will be trained to predict the next note. Finally, music is generated by feeding a random string of notes to the RNN and having it iteratively predict the next note to form a song one note at a time.

In [11]:
# Imports
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.models import load_model
from keras.callbacks import Callback, ModelCheckpoint
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import CuDNNLSTM, LSTM, Bidirectional
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, History

## Loading and Cleaning the Data

First, I will load all the notes from the midi files

In [12]:
def get_notes(path):
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in glob.glob(path + "*.mid"):
        print("Parsing %s" % file)
        
        song = []
        midi = converter.parse(file)
        
        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
            
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                song.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                song.append('.'.join(str(n) for n in element.normalOrder))
        notes.append(song)

    return notes

""" Train a Neural Network to generate music """
# Get notes from midi files
input_dir_choice = 0
input_dir_names = ["Pokemon", "LoZ", "Pokemon GSC"]

input_path = "../" + input_dir_names[input_dir_choice] + " MIDIs/"
notes = get_notes(input_path)

Parsing ../Pokemon MIDIs\Pokemon - Farewell, Pikachu!.mid
Parsing ../Pokemon MIDIs\Pokemon - Lavender town.mid
Parsing ../Pokemon MIDIs\Pokemon - Littleroot Town.mid
Parsing ../Pokemon MIDIs\Pokemon - Lugias Song.mid
Parsing ../Pokemon MIDIs\Pokemon - Oracion.mid
Parsing ../Pokemon MIDIs\Pokemon - Pallet Town.mid
Parsing ../Pokemon MIDIs\Pokemon - Pkmn Elite 4.mid
Parsing ../Pokemon MIDIs\Pokemon - pokecentre theme.mid
Parsing ../Pokemon MIDIs\Pokemon - Pokemon Center Theme.mid
Parsing ../Pokemon MIDIs\Pokemon - Pokemon Center.mid
Parsing ../Pokemon MIDIs\Pokemon - Pokemon Johto - Opening.mid
Parsing ../Pokemon MIDIs\Pokemon - The Ghost at Maiden's Peak.mid
Parsing ../Pokemon MIDIs\Pokemon Black & White - Village Bridge.mid
Parsing ../Pokemon MIDIs\Pokemon Black 2White 2 - Cave of Being.mid
Parsing ../Pokemon MIDIs\Pokemon Black 2White 2 - Join Avenue  Greeting.mid
Parsing ../Pokemon MIDIs\Pokemon Black 2White 2 - Ns Room.mid
Parsing ../Pokemon MIDIs\Pokemon Black 2White 2 - Ns Theme.m

Parsing ../Pokemon MIDIs\Pokemon GoldSilverCrystal - Johto Gym Leader Battle.mid
Parsing ../Pokemon MIDIs\Pokemon GoldSilverCrystal - Johto Wild Pokemon Battle.mid
Parsing ../Pokemon MIDIs\Pokemon GoldSilverCrystal - Kanto Gym Leader Battle.mid
Parsing ../Pokemon MIDIs\Pokemon GoldSilverCrystal - Lavender Town.mid
Parsing ../Pokemon MIDIs\Pokemon GoldSilverCrystal - Menu.mid
Parsing ../Pokemon MIDIs\Pokemon GoldSilverCrystal - National Park.mid
Parsing ../Pokemon MIDIs\Pokemon GoldSilverCrystal - New Bark Town.mid
Parsing ../Pokemon MIDIs\Pokemon GoldSilverCrystal - Olivine Lighthouse.mid
Parsing ../Pokemon MIDIs\Pokemon GoldSilverCrystal - Pokemon Lullaby.mid
Parsing ../Pokemon MIDIs\Pokemon GoldSilverCrystal - Pokemon March.mid
Parsing ../Pokemon MIDIs\Pokemon GoldSilverCrystal - Professor Elms Lab.mid
Parsing ../Pokemon MIDIs\Pokemon GoldSilverCrystal - Professor Oaks Theme.mid
Parsing ../Pokemon MIDIs\Pokemon GoldSilverCrystal - Rival Battle.mid
Parsing ../Pokemon MIDIs\Pokemon Gol

Parsing ../Pokemon MIDIs\Pokemon Pinball - Field Select.mid
Parsing ../Pokemon MIDIs\Pokemon Pinball - Pokedex.mid
Parsing ../Pokemon MIDIs\Pokemon Pinball - Red Field Theme.mid
Parsing ../Pokemon MIDIs\Pokemon Ranger - Ranger Base.mid
Parsing ../Pokemon MIDIs\Pokemon RedBlueYellow - Celadon City.mid
Parsing ../Pokemon MIDIs\Pokemon RedBlueYellow - Cerulean City.mid
Parsing ../Pokemon MIDIs\Pokemon RedBlueYellow - Champion Battle.mid
Parsing ../Pokemon MIDIs\Pokemon RedBlueYellow - Cinnabar Island.mid
Parsing ../Pokemon MIDIs\Pokemon RedBlueYellow - Cycling.mid
Parsing ../Pokemon MIDIs\Pokemon RedBlueYellow - Elite Four.mid
Parsing ../Pokemon MIDIs\Pokemon RedBlueYellow - Ending Theme.mid
Parsing ../Pokemon MIDIs\Pokemon RedBlueYellow - Evolution.mid
Parsing ../Pokemon MIDIs\Pokemon RedBlueYellow - Game Corner Theme.mid
Parsing ../Pokemon MIDIs\Pokemon RedBlueYellow - Gym Leader Battle.mid
Parsing ../Pokemon MIDIs\Pokemon RedBlueYellow - Gym.mid
Parsing ../Pokemon MIDIs\Pokemon RedBlue

Next, I will find all possible notes and use this to determine how to alter the data to a machine readable format.

In [13]:
possibleNotes = set([item for sublist in notes for item in sublist])

n_vocab = len(possibleNotes)
possibleNotes

{'0',
 '0.1',
 '0.1.3',
 '0.1.4',
 '0.1.5',
 '0.1.5.8',
 '0.1.6',
 '0.2',
 '0.2.3',
 '0.2.4',
 '0.2.4.5',
 '0.2.4.7',
 '0.2.4.7.9',
 '0.2.5',
 '0.2.5.7',
 '0.2.5.8',
 '0.2.6',
 '0.2.7',
 '0.3',
 '0.3.4',
 '0.3.5',
 '0.3.5.7',
 '0.3.5.8',
 '0.3.6',
 '0.3.6.7',
 '0.3.6.8',
 '0.3.6.9',
 '0.3.7',
 '0.4',
 '0.4.5',
 '0.4.5.7',
 '0.4.6',
 '0.4.7',
 '0.4.8',
 '0.5',
 '0.5.6',
 '0.6',
 '1',
 '1.2',
 '1.2.4',
 '1.2.4.6',
 '1.2.5',
 '1.2.5.8',
 '1.2.6',
 '1.2.6.9',
 '1.2.7',
 '1.3',
 '1.3.4',
 '1.3.4.6.9',
 '1.3.4.7',
 '1.3.5',
 '1.3.5.7',
 '1.3.5.8',
 '1.3.6',
 '1.3.6.9',
 '1.3.8',
 '1.4',
 '1.4.5',
 '1.4.6',
 '1.4.6.9',
 '1.4.7',
 '1.4.7.9',
 '1.4.7.9.10',
 '1.4.8',
 '1.5',
 '1.5.6',
 '1.5.7',
 '1.5.8',
 '1.5.8.9',
 '1.5.9',
 '1.6',
 '1.6.7',
 '1.7',
 '10',
 '10.0',
 '10.0.1',
 '10.0.2',
 '10.0.2.4.7',
 '10.0.2.5',
 '10.0.3',
 '10.0.3.5',
 '10.0.4',
 '10.0.4.5',
 '10.0.5',
 '10.1',
 '10.1.2',
 '10.1.2.5',
 '10.1.3',
 '10.1.3.6',
 '10.1.4',
 '10.1.4.5.6',
 '10.1.4.6',
 '10.1.5',
 '10.11',
 '10.

Now I will prepare the sequences of notes by looking at each song individually. I will first grab an arrays of size **sequence_length** with a stride of **step_size** from each song. Then I will map the chords to integers so the model can learn from that and normalize the input between 0-1.

In [14]:
def prepare_sequences(notes, allNotes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100
    step_size = 20

    # get all pitch names
    pitchnames = sorted(allNotes)

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for song in notes:
        for i in range(0, len(song) - sequence_length, step_size):
            sequence_in = song[i:i + sequence_length]
            sequence_out = song[i + sequence_length]
            network_input.append([note_to_int[char] for char in sequence_in])
            network_output.append(note_to_int[sequence_out])

    # reshape the input into a format compatible with LSTM layers
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # normalize input between 0 and 1
    network_input = network_input / float(n_vocab)

    # Make one-hot-encoding
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

network_input, network_output = prepare_sequences(notes, possibleNotes, n_vocab)

## Constructing the model

I will now construct the model using CuDNNLSTM cells because they are significantly faster than regular LSTM cells due to being optimized for CuDA. I will have two CuDNNLSTM layers, followed by two dense layers and a final softmax activation layer to output the most probable result.

Hyperparameters:
* Optimizer - ADAM because it is considered one of the best
* Loss - categorical_crossentropy because it penalizes wrong predictions of multi-class problems best
* Epochs - More epochs are generally better as long as they don't overfit. I track loss over time and have checkpoints every 5 epochs so this will not be a problem
* Batch Size - This determines how many instances should be considered in each batch. Realistically, each different song would interfere with the other so I will reduce this.

In [23]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(CuDNNLSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
    model.add(Dropout(0.3))
    model.add(Bidirectional(CuDNNLSTM(512)))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab-1))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

# Set up the model
model = create_network(network_input, n_vocab)
history = History()

# Save on each epoch (because training isn't cheap!!!)
outputDest = '../output/LSTM_' + input_dir_names[input_dir_choice] + '_' + str(int(time.time())) + '/'
if not os.path.exists(outputDest):
    os.makedirs(outputDest)

cp_callback = ModelCheckpoint(filepath=outputDest + "LSTMmodel_weights_{epoch:02d}.hdf5",
                              save_weights_only=True,
                              verbose=1,
                              period=5)

# Set parameters
n_epochs = 200
batch_size = 10
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_5 (CuDNNLSTM)     (None, 100, 256)          265216    
_________________________________________________________________
dropout_5 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 512)               1052672   
_________________________________________________________________
dense_5 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 496)               63984     
_________________________________________________________________
activation_3 (Activation)    (None, 496)              

## Training the Model

I will save the final model, but keep checkpoints along the way to avoid overfitting and also use these to generate different midis.

In [24]:
model.fit(network_input, network_output, callbacks=[history, cp_callback], epochs=n_epochs, batch_size=batch_size)
model.save(outputDest + 'LSTMmodel_final.h5')

Epoch 1/200
6828/6828 [==============================] - 32s 5ms/step - loss: 4.9821
Epoch 2/200
6828/6828 [==============================] - 31s 5ms/step - loss: 4.8099
Epoch 3/200
6828/6828 [==============================] - 31s 4ms/step - loss: 4.7731
Epoch 4/200
6828/6828 [==============================] - 31s 5ms/step - loss: 4.7533
Epoch 5/200
6828/6828 [==============================] - 32s 5ms/step - loss: 4.7373

Epoch 00005: saving model to ../output/LSTM_Pokemon_1570065687/LSTMmodel_weights_05.hdf5
Epoch 6/200
6828/6828 [==============================] - 32s 5ms/step - loss: 4.7218
Epoch 7/200
6828/6828 [==============================] - 31s 5ms/step - loss: 4.7022
Epoch 8/200
6828/6828 [==============================] - 31s 4ms/step - loss: 4.6853
Epoch 9/200
6828/6828 [==============================] - 31s 4ms/step - loss: 4.6695
Epoch 10/200
6828/6828 [==============================] - 30s 4ms/step - loss: 4.6560

Epoch 00010: saving model to ../output/LSTM_Pokemon_157006

Epoch 76/200
6828/6828 [==============================] - 30s 4ms/step - loss: 0.3800
Epoch 77/200
6828/6828 [==============================] - 30s 4ms/step - loss: 0.3653
Epoch 78/200
6828/6828 [==============================] - 30s 4ms/step - loss: 0.3688
Epoch 79/200
6828/6828 [==============================] - 30s 4ms/step - loss: 0.3548
Epoch 80/200
6828/6828 [==============================] - 30s 4ms/step - loss: 0.3141

Epoch 00080: saving model to ../output/LSTM_Pokemon_1570065687/LSTMmodel_weights_80.hdf5
Epoch 81/200
6828/6828 [==============================] - 30s 4ms/step - loss: 0.3334: 0s - los
Epoch 82/200
6828/6828 [==============================] - 31s 4ms/step - loss: 0.3423
Epoch 83/200
6828/6828 [==============================] - 30s 4ms/step - loss: 0.2875
Epoch 84/200
6828/6828 [==============================] - 30s 4ms/step - loss: 0.3336: 0s - lo
Epoch 85/200
6828/6828 [==============================] - 30s 4ms/step - loss: 0.2914

Epoch 00085: saving model to .

6828/6828 [==============================] - 30s 4ms/step - loss: 0.1395
Epoch 154/200
6828/6828 [==============================] - 30s 4ms/step - loss: 0.1350
Epoch 155/200
6828/6828 [==============================] - 30s 4ms/step - loss: 0.1229

Epoch 00155: saving model to ../output/LSTM_Pokemon_1570065687/LSTMmodel_weights_155.hdf5
Epoch 156/200
6828/6828 [==============================] - 30s 4ms/step - loss: 0.1270
Epoch 157/200
6828/6828 [==============================] - 30s 4ms/step - loss: 0.1351
Epoch 158/200
6828/6828 [==============================] - 30s 4ms/step - loss: 0.1217
Epoch 159/200
6828/6828 [==============================] - 30s 4ms/step - loss: 0.1297
Epoch 160/200
6828/6828 [==============================] - 30s 4ms/step - loss: 0.1319:

Epoch 00160: saving model to ../output/LSTM_Pokemon_1570065687/LSTMmodel_weights_160.hdf5
Epoch 161/200
6828/6828 [==============================] - 30s 4ms/step - loss: 0.1514
Epoch 162/200
6828/6828 [=======================

In [25]:
# Plot the model losses
pd.DataFrame(history.history).plot()
plt.savefig(outputDest + 'LSTM_Loss_per_Epoch.png', transparent=True)
plt.close()

## Generating Music

I will now use the model to generate music by feeding it a random string of notes and have it predict the next one, then have it predict the one after that until a full song has been generated.

In [19]:
def generate_notes(model, notes, network_input, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    pitchnames = sorted(notes)
    
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = np.append(pattern,index)
        pattern = pattern[1:len(pattern)]
    
    print(prediction_output)

    return prediction_output

prediction_output = generate_notes(model, possibleNotes, network_input, n_vocab)

Next, I will create a midi using these notes and save to a file

In [20]:
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))
    
create_midi(prediction_output, outputDest + 'LSTM_output')

Alternatively, I can run this script to convert all of the models into midi files and select my favourite from a much larger album.

In [26]:
# Have each model make a song
count = 0
for model_path in glob.glob(outputDest + "*.hdf5"):
    print("Composing from %s" % model_path)
    model.load_weights(model_path)
    prediction_output = generate_notes(model, possibleNotes, network_input, n_vocab)
    create_midi(prediction_output, outputDest + 'LSTM_output_' + str(count))
    print(outputDest + 'LSTM_output_' + str(count))
    count += 1

Composing from ../output/LSTM_Pokemon_1570065687\LSTMmodel_weights_05.hdf5
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',

../output/LSTM_Pokemon_1570065687/LSTM_output_2
Composing from ../output/LSTM_Pokemon_1570065687\LSTMmodel_weights_105.hdf5
['11.4', '2', '2', '2', '2', '11.4', '11.4', '11.4', '2', '2', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '11.4', '3.8', '11.4', '3.8', '11.4', 'E4', 'A3', 'D2', 'E4', 'E4', 'E4', 'A3', 'A3', 'A3', 'A3', 'E4', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'C#4', 'A3', 'E4', 'A3', 'A3', 'C#4', 'C#4', 'A3', 'A3', 'A3', 'A3', 'A3', 'A4', 'C4', 'C4', 'C4', '2.6', 'C#4', 'C4', 'A3', 'A3', 'D3', 'G#3', 'D2', 'C4', 'C4', 'C4', 'A4', 'A4', 'A4', 'A4', 'C4', 'C4', 'C4', 'A3', 'A3', 'A4', 'C4', 'C4', 'C4', 'C4', 'C#5', 'A4', 

../output/LSTM_Pokemon_1570065687/LSTM_output_5
Composing from ../output/LSTM_Pokemon_1570065687\LSTMmodel_weights_120.hdf5
['A3', 'A3', 'A3', 'F3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'E3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'E3', 'E3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'B-3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'B-3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'G6', 'A3', 'G6', 'B3', 'G6', 'E4', 'E4', 'E6', 'B-3', 'E4', 'B-1', 'F3', 'F#3', 'A3', 'C#4', 'F2', 'F2', 'C3', 'G5', 'G#3', 'C#3', 'C2', 'G#3', 'G#4', 'C6', 'E4', 'G#3', 'G#3', 'B-4', 'B-4', 'B-4', 'B-4', 'B-4', 'C#3', 'E2', 'C3', 'E-4', 'B4', 'G2', 'B-2', 'C#3', 'B-2', 'C#3', 'A4', 'A4', 'C3', 'C3', 'C3', 'A3', 'C3', 'B-2', 'B-2', 'B-2', 'C#3', 'B-2', 'A2', 'B-2', 'A4', 'B3', 'C3', 'C3', 

../output/LSTM_Pokemon_1570065687/LSTM_output_8
Composing from ../output/LSTM_Pokemon_1570065687\LSTMmodel_weights_135.hdf5
['A3', 'E4', '0.3', 'C5', 'A3', 'E4', 'E4', 'E4', 'E4', 'A3', 'A3', 'A3', 'A3', 'A3', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'A3', 'A3', 'A3', 'A3', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'A3', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'A3', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'A3', 'E-5', 'F5', 'F4', 'B-1', 'E-4', 'C#4', 'A2', 'F4', 'F#5', 'B-5', 'A2', 'B-5', 'A2', 'A2', 'B4', 'A5', 'A5', 'F#3', 'G4', 'A1', 'A2', 'F#5', 'E-2', 'D5', 'F#5', 'F2', 'D2', 'B-2', 'D4', 'D4', 'C#3', 'C4', 'G5', 'F4', 'D2', 'E-4', 'B4', 'E4', 'B-2', 'F#2', 'F#4', 'F#4', 'A2', 'A2', 'A2', '11', '4.8', '9'

['2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5'

...

../output/LSTM_Pokemon_1570065687/LSTM_output_26
Composing from ../output/LSTM_Pokemon_1570065687\LSTMmodel_weights_35.hdf5
['11.4', '2', '2', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '2.5', '2.5', '5.8.0', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '11.4', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '5.8.0', '4.7', '2.5', '2.5', '5.8.0', '11.4', '2.5', '5.8.0', '11.4', '2.5', '5.8.0', '11.4', '2.5', '5.8.0', '11.4', '2.5', '11.4', '5.8.0', '11.4', '2.5', '11.4', '5.8.0', '5.8.0', '4.7', '2.5', '2.5', '5.8.0', '11.4', '2.5', '5.8.0', '11.4', '2.5', '5.8.0', '11.4', '2.5', '11.4', '2.5', '11.4', '2.5', '11.4', '11.4', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5',

../output/LSTM_Pokemon_1570065687/LSTM_output_29
Composing from ../output/LSTM_Pokemon_1570065687\LSTMmodel_weights_50.hdf5
['11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '3.8', '11.4', 'F#6', '3.8', '11.4', '5.9.0', '11.4', '3.6', '11.4', 'E4', '11.4', '7.10', '3.8', '3.6', '11.4', 'A3', 'E4', 'E4', 'E4', 'A3', 'A3', '11.4', '11.4', 'G#5', 'E4', 'E4', '4', 'A3', 'F#3', 'A3', 'C5', 'C5', 'B4', 'B4', 'B4', 'A3', 'A3', 'A3', 'E4', 'E4', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'A3', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'G6', 'A3', 'A3', 'E4', 'E4', 'E4', 'E4', 'E4', 'F#4', 'F#4', 'F#4', 'B-1', 'B-1', 'B-1', '1.2', '10.2', '10.2', 'E3', 'F#5', 'F#5', 'E3', 'C#4', 'D5', 'A5', 'B4', '0.3.7', '4.8', 'C4', 'C#3', 'B-1', 'A3', 'F#2', 'D5', '4', 'E2', 'G4', 'E5', 'B-1

../output/LSTM_Pokemon_1570065687/LSTM_output_32
Composing from ../output/LSTM_Pokemon_1570065687\LSTMmodel_weights_65.hdf5
['A3', '3.8', 'D2', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'B3', 'F3', 'B-3', 'A3', 'A3', 'A3', 'B3', 'A4', 'A3', 'B2', 'A5', 'A5', 'B3', 'B3', 'B3', 'A3', 'B2', 'B3', 'B3', 'B3', 'B3', 'A3', 'A3', 'B3', 'B3', 'B3', 'B3', 'A3', 'A3', 'B3', 'B3', 'B3', 'A3', 'A3', 'A3', 'A3', 'B3', 'B3', 'A3', 'A3', 'A3', 'B3', 'B3', 'B3', 'B3', 'A3', 'A3', 'B3', 'B3', 'B3', 'B3

../output/LSTM_Pokemon_1570065687/LSTM_output_35
Composing from ../output/LSTM_Pokemon_1570065687\LSTMmodel_weights_80.hdf5
['0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0.4.7', '0

../output/LSTM_Pokemon_1570065687/LSTM_output_38
Composing from ../output/LSTM_Pokemon_1570065687\LSTMmodel_weights_95.hdf5
['11.4', '2', '11.4', '11.4', '0.4.7', '0.4.7', '11.4', '0.4.7', '0.4.7', '11.4', '0.4.7', '0.4.7', '11.4', '11.4', '6.9', '6.9', '0.4.7', '0.4.7', '11.4', '11.4', 'B4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'B4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'B4', 'B4', 'B4', 'B4', 'B4', 'E4', 'B4', 'E4', 'E4', 'E4', 'E4', 'E4', 'F#4', '5.10', 'D4', 'G6', 'C3', 'C3', 'C3', 'G6', 'B-3', 'G6', 'D4', 'B-3', 'C5', 'C5', 'D4', 'D4', 'D4', 'B3', 'B2', 'B3', 'B3', 'E-5', 'D2', 'C#3', 'C#5', 'A2', 'A2', 'A2', 'F5', 'F#2', 'F#2', 'A2', 'F#3', 'A2', 'D4', 'G4', 'A3', 'A3', 'A2', 'A4', 'A4', 'B4',